#GPU Accelerated Principal Component Analysis (PCA) using RAPIDS on a Sample Dataset with CPU vs GPU comparison

#### Verifying GPUs

RAPIDS requires GPUs with Pascal Architecture or better. That means any GPUs starting with K (Kepler) series (e.g. K80) or M (Maxwell) will not work with RAPIDS. You can use the `nvidia-smi` command to verify the type of your GPU as well as the memory size which may be needed for some of the RAPIDS examples.

In [3]:
%sh nvidia-smi

Tue Mar 19 01:56:45 2019 
+-----------------------------------------------------------------------------+
 NVIDIA-SMI 396.44 Driver Version: 396.44 |
-------------------------------+----------------------+----------------------+
 GPU Name Persistence-M| Bus-Id Disp.A | Volatile Uncorr. ECC |
 Fan Temp Perf Pwr:Usage/Cap| Memory-Usage | GPU-Util Compute M. |
===============================+======================+======================|
 0 Tesla V100-SXM2... Off | 00000000:00:1E.0 Off | 0 |
 N/A 33C P0 23W / 300W | 0MiB / 16160MiB | 0% Default |
+-------------------------------+----------------------+----------------------+
 
+-----------------------------------------------------------------------------+
 Processes: GPU Memory |
 GPU PID Type Process name Usage |
=============================================================================|
 No running processes found |
+-----------------------------------------------------------------------------+

## Let's begin by importing RAPIDS and scikit learn libraries!

In [5]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA as skPCA
from cuml import PCA as cumlPCA
import cudf
import os

## Downloading the data
For this example we are downloading a sample dataset (Mortgage.csv) from Nvidia's repository

__We have already completed Data prep (ETL) and feature engineering on this dataset and the dataset is ready for Machine Learning__

__Note:__ If you already have a dataset, please don't run the next command

In [7]:
%sh 

wget https://github.com/rapidsai/notebooks-extended/raw/master/data/mortgage_data_tar.tar.gz

tar -xzf mortgage_data_tar.tar.gz

wget: /databricks/python/lib/libcrypto.so.1.0.0: no version information available (required by wget)
wget: /databricks/python/lib/libssl.so.1.0.0: no version information available (required by wget)
wget: /databricks/python/lib/libssl.so.1.0.0: no version information available (required by wget)
--2019-03-19 01:58:12-- https://github.com/rapidsai/notebooks-extended/raw/master/data/mortgage_data_tar.tar.gz
Resolving github.com (github.com)... 192.30.255.113, 192.30.255.112
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/rapidsai/notebooks-extended/master/data/mortgage_data_tar.tar.gz [following]
--2019-03-19 01:58:12-- https://raw.githubusercontent.com/rapidsai/notebooks-extended/master/data/mortgage_data_tar.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.52.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 77774776 (74M) [application/octet-stream]
Saving to: ‘mortgage_data_tar.tar.gz’

 0K .......... .......... .......... .......... .......... 0% 3.62M 20s
 50K .......... .......... .......... .......... .......... 0% 7.08M 15s
 100K .......... .......... .......... .......... .......... 0% 7.16M 14s
 150K .......... .......... .......... .......... .......... 0% 108M 10s
 200K .......... .......... .......... .......... .......... 0% 236M 8s
 250K .......... .......... .......... .......... .......... 0% 8.19M 9s
 300K .......... .......... .......... .......... .......... 0% 71.6M 7s
 350K .......... .......... .......... .......... .......... 0% 201M 7s
 400K .......... .......... .......... .......... .......... 0% 234M 6s
 450K .......... .......... .......... .......... .......... 0% 7.70M 6s
 500K .......... .......... .......... .......... .......... 0% 216M 6s
 550K .......... .......... .......... .......... .......... 0% 198M 5s
 600K .......... .......... .......... .......... .......... 0% 243M 5s
 650K .......... .......... .......... .......... .......... 0% 235M 5s
 700K .......... .......... .......... .......... .......... 0% 240M 4s
 750K .......... .......... .......... .......... .......... 1% 210M 4s
 800K .......... .......... .......... .......... .......... 1% 72.0M 4s
 850K .......... .......... .......... .......... .......... 1% 10.4M 4s
 900K .......... .......... .......... .......... .......... 1% 147M 4s
 950K .......... .......... .......... .......... .......... 1% 201M 4s
 1000K .......... .......... .......... .......... .......... 1% 234M 3s
 1050K .......... .......... .......... .......... .......... 1% 28.3M 3s
 1100K .......... .......... .......... .......... .......... 1% 12.4M 4s
 1150K .......... .......... .......... .......... .......... 1% 120M 3s
 1200K .......... .......... .......... .......... .......... 1% 154M 3s
 1250K .......... .......... .......... .......... .......... 1% 231M 3s
 1300K .......... .......... .......... .......... .......... 1% 26.0M 3s
 1350K .......... .......... .......... .......... .......... 1% 12.1M 3s
 1400K .......... .......... .......... .......... .......... 1% 145M 3s
 1450K .......... .......... .......... .......... .......... 1% 139M 3s
 1500K .......... .......... .......... .......... .......... 2% 232M 3s
 1550K .......... .......... .......... .......... .......... 2% 25.8M 3s
 1600K .......... .......... .......... .......... .......... 2% 217M 3s
 1650K .......... .......... .......... .......... .......... 2% 12.0M 3s
 1700K .......... .......... .......... .......... .......... 2% 147M 3s
 1750K .......... .......... .......... .......... .......... 2% 195M 3s
 1800K .......... .......... .......... .......... .......... 2% 24.7M 3s
 1850K .......... .......... .......... .......... .......... 2% 232M 3s
 1900K .......... .......... .......... .......... .......... 2% 12

Check that everything downloaded and extracted ok

In [9]:
%sh

ls 

acq
mortgage.csv
mortgage_data.avro
mortgage.npy
names.csv
perf

Make a directory and copy the extracted file there.

In [11]:
%sh

mkdir -p /dbfs/RAPIDS/mortgage
rm -rf /dbfs/RAPIDS/mortgage/mortgage_data.avro
mv mortgage_data.avro /dbfs/RAPIDS/mortgage/mortgage_data.avro

## Loading the data with Spark

In [13]:
data = spark.read.format("avro").load("/RAPIDS/mortgage/mortgage_data.avro/")

__Helper Functions to remove any null values__

In [15]:
from pyspark.sql.types import DoubleType

def recast(df):
   for column, data_type in df.dtypes:
       if str(data_type) == "string":
           df = df.withColumn(column, df[column].cast("float"))
   return df

data = recast(data).fillna(-1)

__Let's check out the dataset in the spark dataframe and count the number of rows in the dataset__

In [17]:
display(data)

_c0,_c1,_c2,_c3,_c4,_c5,_c6,_c7,_c8,_c9,_c10,_c11,_c12,_c13,_c14,_c15,_c16,_c17,_c18,_c19,_c20,_c21,_c22,_c23,_c24,_c25,_c26,_c27,_c28,_c29,_c30,_c31,_c32,_c33,_c34,_c35,_c36,_c37,_c38,_c39,_c40,_c41,_c42,_c43,_c44,_c45,_c46,_c47,_c48,_c49,_c50,_c51,_c52,_c53,_c54,_c55,_c56,_c57,_c58,_c59,_c60,_c61,_c62,_c63,_c64,_c65,_c66,_c67,_c68,_c69,_c70,_c71,_c72,_c73,_c74,_c75,_c76,_c77,_c78,_c79,_c80,_c81,_c82,_c83,_c84,_c85,_c86,_c87,_c88,_c89,_c90,_c91,_c92,_c93,_c94,_c95,_c96,_c97,_c98,_c99,_c100,_c101,_c102,_c103,_c104,_c105,_c106,_c107,_c108,_c109,_c110,_c111,_c112,_c113,_c114,_c115,_c116,_c117,_c118,_c119,_c120,_c121,_c122,_c123,_c124,_c125,_c126,_c127,_c128,_c129,_c130,_c131,_c132,_c133,_c134,_c135,_c136,_c137,_c138,_c139,_c140,_c141,_c142,_c143,_c144,_c145,_c146,_c147,_c148,_c149,_c150,_c151,_c152,_c153,_c154,_c155,_c156,_c157,_c158,_c159,_c160,_c161,_c162,_c163,_c164,_c165,_c166,_c167,_c168,_c169,_c170,_c171,_c172,_c173,_c174,_c175,_c176,_c177,_c178,_c179,_c180,_c181,_c182,_c183,_c184,_c185,_c186,_c187,_c188,_c189,_c190,_c191,_c192,_c193,_c194,_c195,_c196,_c197,_c198,_c199,_c200,_c201,_c202,_c203,_c204,_c205,_c206,_c207,_c208,_c209,_c210,_c211,_c212,_c213,_c214,_c215,_c216,_c217,_c218,_c219,_c220,_c221,_c222,_c223,_c224,_c225,_c226,_c227,_c228,_c229,_c230,_c231,_c232,_c233,_c234,_c235,_c236,_c237,_c238,_c239,_c240,_c241,_c242,_c243,_c244,_c245,_c246,_c247,_c248,_c249,_c250,_c251,_c252,_c253,_c254,_c255,_c256,_c257,_c258,_c259,_c260,_c261,_c262,_c263,_c264,_c265,_c266,_c267,_c268,_c269,_c270,_c271,_c272,_c273,_c274,_c275,_c276,_c277,_c278,_c279,_c280,_c281,_c282,_c283,_c284,_c285,_c286,_c287,_c288,_c289,_c290,_c291,_c292,_c293,_c294,_c295,_c296,_c297,_c298,_c299,_c300,_c301,_c302,_c303,_c304,_c305,_c306,_c307,_c308,_c309,_c310,_c311,_c312,_c313,_c314,_c315,_c316,_c317,_c318,_c319,_c320,_c321,_c322,_c323,_c324,_c325,_c326,_c327,_c328,_c329,_c330,_c331,_c332,_c333,_c334,_c335,_c336,_c337,_c338,_c339,_c340,_c341,_c342,_c343,_c344,_c345,_c346,_c347,_c348,_c349,_c350,_c351,_c352,_c353,_c354,_c355,_c356,_c357,_c358,_c359,_c360,_c361,_c362,_c363,_c364,_c365,_c366,_c367,_c368,_c369,_c370,_c371,_c372,_c373,_c374,_c375,_c376,_c377,_c378,_c379,_c380,_c381,_c382,_c383,_c384,_c385,_c386,_c387,_c388,_c389,_c390,_c391,_c392,_c393,_c394,_c395,_c396,_c397,_c398,_c399,_c400,_c401,_c402,_c403,_c404,_c405,_c406,_c407,_c408,_c409,_c410,_c411,_c412,_c413,_c414,_c415,_c416,_c417,_c418,_c419,_c420,_c421,_c422,_c423,_c424,_c425,_c426,_c427,_c428,_c429,_c430,_c431,_c432,_c433,_c434,_c435,_c436,_c437,_c438,_c439,_c440,_c441,_c442,_c443,_c444,_c445,_c446,_c447,_c448,_c449,_c450,_c451,_c452,_c453,_c454,_c455,_c456,_c457,_c458,_c459,_c460,_c461,_c462,_c463,_c464,_c465,_c466,_c467,_c468,_c469,_c470,_c471,_c472,_c473,_c474,_c475,_c476,_c477,_c478,_c479,_c480,_c481,_c482,_c483,_c484,_c485,_c486,_c487,_c488,_c489,_c490,_c491,_c492,_c493,_c494,_c495,_c496,_c497,_c498,_c499,_c500,_c501,_c502,_c503,_c504,_c505,_c506,_c507,_c508,_c509,_c510,_c511,_c512,_c513,_c514,_c515,_c516,_c517,_c518,_c519,_c520,_c521,_c522,_c523,_c524,_c525,_c526,_c527,_c528,_c529,_c530,_c531,_c532,_c533,_c534,_c535,_c536,_c537,_c538,_c539,_c540,_c541
0.5416667,0.06061276,0.4144144,0.234375,0.10555556,0.0,0.0,0.04448628,0.05995058,0.061697926,0.7163188,0.08303197,0.23162445,0.11361196,0.21241173,0.006779519,0.0,0.5,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0

In [18]:
dataCount = data.count() # We're storing this value for later use
print(dataCount)

800000

__Helper Functions to compare CPU vs GPU results__

In [20]:
from sklearn.metrics import mean_squared_error
def array_equal(a,b,threshold=2e-3,with_sign=True):
    a = to_nparray(a)
    b = to_nparray(b)
    if with_sign == False:
        a,b = np.abs(a),np.abs(b)
    error = mean_squared_error(a,b)
    res = error<threshold
    return res

def to_nparray(x):
    if isinstance(x,np.ndarray) or isinstance(x,pd.DataFrame):
        return np.array(x)
    elif isinstance(x,np.float64):
        return np.array([x])
    elif isinstance(x,cudf.DataFrame) or isinstance(x,cudf.Series):
        return x.to_pandas().values
    return x    

## Converting the Spark Dataframe into Pandas Dataframe

__Load data function allows you to create a user defined sample of your data and converts the spark dataframe to pandas dataframe.  Then, it removes any null values in the dataset.  If you want to to experiment with a different dataset sizes, use the random array generator to load the random data.__

In [23]:
def load_data(nrows, ncols):
  try:
    frac = nrows/dataCount # as sample() takes an integer, we are creating a factor by which to get the approximate number of rows 
    print(frac) # just for checks :)
    if (frac > 1): 
      frac = 1.0
    print(frac) # just for checks++ :)
    X = data.sample(True, frac) 
    print(X)
    df = X.toPandas() # we then convert the Spark Dataframe to Pandas.  
    print("everything worked")
  except Exception as e: 
    print(e)
    print('use random data')
    X = np.random.rand(nrows,ncols)
    df = pd.DataFrame({'fea%d'%i:X[:,i] for i in range(X.shape[1])})
    print("only random data")
  return df

__Setting up data in Pandas Dataframe using Load data and null workaround function__

In [25]:
%%time
nrows = 2**20
nrows = int(nrows * 1.5)
ncols = 400

X = load_data(nrows,ncols)

1.96608
1.0
DataFrame[_c0: float, _c1: float, _c2: float, _c3: float, _c4: float, _c5: float, _c6: float, _c7: float, _c8: float, _c9: float, _c10: float, _c11: float, _c12: float, _c13: float, _c14: float, _c15: float, _c16: float, _c17: float, _c18: float, _c19: float, _c20: float, _c21: float, _c22: float, _c23: float, _c24: float, _c25: float, _c26: float, _c27: float, _c28: float, _c29: float, _c30: float, _c31: float, _c32: float, _c33: float, _c34: float, _c35: float, _c36: float, _c37: float, _c38: float, _c39: float, _c40: float, _c41: float, _c42: float, _c43: float, _c44: float, _c45: float, _c46: float, _c47: float, _c48: float, _c49: float, _c50: float, _c51: float, _c52: float, _c53: float, _c54: float, _c55: float, _c56: float, _c57: float, _c58: float, _c59: float, _c60: float, _c61: float, _c62: float, _c63: float, _c64: float, _c65: float, _c66: float, _c67: float, _c68: float, _c69: float, _c70: float, _c71: float, _c72: float, _c73: float, _c74: float, _c75: float, _c76: float, _c77: float, _c78: float, _c79: float, _c80: float, _c81: float, _c82: float, _c83: float, _c84: float, _c85: float, _c86: float, _c87: float, _c88: float, _c89: float, _c90: float, _c91: float, _c92: float, _c93: float, _c94: float, _c95: float, _c96: float, _c97: float, _c98: float, _c99: float, _c100: float, _c101: float, _c102: float, _c103: float, _c104: float, _c105: float, _c106: float, _c107: float, _c108: float, _c109: float, _c110: float, _c111: float, _c112: float, _c113: float, _c114: float, _c115: float, _c116: float, _c117: float, _c118: float, _c119: float, _c120: float, _c121: float, _c122: float, _c123: float, _c124: float, _c125: float, _c126: float, _c127: float, _c128: float, _c129: float, _c130: float, _c131: float, _c132: float, _c133: float, _c134: float, _c135: float, _c136: float, _c137: float, _c138: float, _c139: float, _c140: float, _c141: float, _c142: float, _c143: float, _c144: float, _c145: float, _c146: float, _c147: float, _c148: float, _c149: float, _c150: float, _c151: float, _c152: float, _c153: float, _c154: float, _c155: float, _c156: float, _c157: float, _c158: float, _c159: float, _c160: float, _c161: float, _c162: float, _c163: float, _c164: float, _c165: float, _c166: float, _c167: float, _c168: float, _c169: float, _c170: float, _c171: float, _c172: float, _c173: float, _c174: float, _c175: float, _c176: float, _c177: float, _c178: float, _c179: float, _c180: float, _c181: float, _c182: float, _c183: float, _c184: float, _c185: float, _c186: float, _c187: float, _c188: float, _c189: float, _c190: float, _c191: float, _c192: float, _c193: float, _c194: float, _c195: float, _c196: float, _c197: float, _c198: float, _c199: float, _c200: float, _c201: float, _c202: float, _c203: float, _c204: float, _c205: float, _c206: float, _c207: float, _c208: float, _c209: float, _c210: float, _c211: float, _c212: float, _c213: float, _c214: float, _c215: float, _c216: float, _c217: float, _c218: float, _c219: float, _c220: float, _c221: float, _c222: float, _c223: float, _c224: float, _c225: float, _c226: float, _c227: float, _c228: float, _c229: float, _c230: float, _c231: float, _c232: float, _c233: float, _c234: float, _c235: float, _c236: float, _c237: float, _c238: float, _c239: float, _c240: float, _c241: float, _c242: float, _c243: float, _c244: float, _c245: float, _c246: float, _c247: float, _c248: float, _c249: float, _c250: float, _c251: float, _c252: float, _c253: float, _c254: float, _c255: float, _c256: float, _c257: float, _c258: float, _c259: float, _c260: float, _c261: float, _c262: float, _c263: float, _c264: float, _c265: float, _c266: float, _c267: float, _c268: float, _c269: float, _c270: float, _c271: float, _c272: float, _c273: float, _c274: float, _c275: float, _c276: float, _c277: float, _c278: float, _c279: float, _c280: float, _c281: float, _c282: float, _c283: float, _c284: float, _c285: float, _c286: float, _c287: float, _c288: float, _c289: float, _c290: float, _c291: float, 

# Brief Intro to PCA parameters

Let's take a look into all possible parameters that we can use when applying PCA: 
http://scikitlearn.org/stable/modules/generated/sklearn.decomposition.PCA.html

We will start here with the following :

__n_components__ : int, float, None or string  
Number of components to keep. if n_components is not set all components are kept

__whiten__ : bool, optional (default False) 
When True (False by default) the components_ vectors are multiplied by the square root of n_samples and then divided by the singular values to ensure uncorrelated outputs with unit component-wise variances. Whitening will remove some information from the transformed signal (the relative variance scales of the components) but can sometime improve the predictive accuracy of the downstream estimators by making their data respect some hard-wired assumptions

__random_state__ : int, RandomState instance or None, optional (default None) 
If int, random_state is the seed used by the random number generator

__svd_solver__ : string {‘auto’, ‘full’, ‘arpack’, ‘randomized’} 
If "full" :run exact full SVD calling the standard LAPACK solver via scipy.linalg.svd and select the components by postprocessing

In [28]:
n_components = 10
whiten = False
random_state = 42
svd_solver="full"


# Run PCA on CPU

Let's check the time needed to execute PCA function using standard sklearn library. 
__Note: this algorithm runs on CPU only.__

In [31]:
import multiprocessing
print(multiprocessing.cpu_count()) # Return the number of CPUs in the system.

In [32]:
%%time
pca_sk = skPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_sk = pca_sk.fit_transform(X)

CPU times: user 5min 27s, sys: 46.7 s, total: 6min 14s
Wall time: 59.5 s

# Run PCA on GPU

Now, before we execute PCA function using RAPIDS cuml library we will first read the data in GPU data format using cudf. 

__cudf__ - GPU DataFrame manipulation library https://github.com/rapidsai/cudf

__cuml__ - suite of libraries that implements a machine learning algorithms within the RAPIDS data science ecosystem https://github.com/rapidsai/cuml

In [35]:
Xt = cudf.DataFrame.from_pandas(X) # Convert Pandas Dataframe to GPU Dataframe!

In [36]:
%%time
pca_cuml = cumlPCA(n_components=n_components,svd_solver=svd_solver, 
            whiten=whiten, random_state=random_state)
result_cuml = pca_cuml.fit_transform(Xt)

CPU times: user 2.21 s, sys: 208 ms, total: 2.42 s
Wall time: 2.41 s

In [37]:
for attr in ['singular_values_','components_','explained_variance_',
             'explained_variance_ratio_']:
    passed = array_equal(getattr(pca_sk,attr),getattr(pca_cuml,attr))
    message = 'compare pca: cuml vs sklearn {:>25} {}'.format(attr,'equal' if passed else 'NOT equal')
    print(message)

compare pca: cuml vs sklearn singular_values_ equal
compare pca: cuml vs sklearn components_ equal
compare pca: cuml vs sklearn explained_variance_ equal
compare pca: cuml vs sklearn explained_variance_ratio_ equal

In [38]:
# Spark ML accelerated with RAPIDS
passed = array_equal(result_sk,result_cuml)
message = 'compare pca: cuml vs sklearn transformed results %s'%('equal'if passed else 'NOT equal')
print(message)

compare pca: cuml vs sklearn transformed results equal